# PHIA Demo - VS Code Version

**Before running:** Add your API keys to `api_keys.py` file

In [ ]:
# Cell 1: Imports and Setup
import os
import sys
import glob
import pandas as pd
import google.generativeai as genai

# Add current directory to path
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Import API keys
from api_keys import GOOGLE_API_KEY, TAVILY_API_KEY

# Import project modules
from onetwo import ot
from onetwo.backends import gemini_api
from data_utils import load_persona
from phia_agent import get_react_agent, QUESTION_PREFIX

print("✓ All modules imported successfully")

In [ ]:
# Cell 2: Configure API Keys
if not GOOGLE_API_KEY:
    print("❌ Please add your Google API key to api_keys.py")
    print("Get it from: https://aistudio.google.com")
else:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✓ Google API key configured")

use_mock_search = not bool(TAVILY_API_KEY)
if use_mock_search:
    print("⚠️ Using mock search (add Tavily key for real web search)")
else:
    print("✓ Tavily API key configured")

In [ ]:
# Cell 3: Setup LLM Backend
llm_engine = gemini_api.GeminiAPI(
    generate_model_name="models/gemini-1.5-pro",
    api_key=GOOGLE_API_KEY,
    temperature=0.0,
)
llm_engine.register()
print("✓ LLM Backend registered")

In [ ]:
# Cell 4: Load Data
summary_path = os.path.join("synthetic_wearable_users", "summary_df_502.csv")
activities_path = os.path.join("synthetic_wearable_users", "exercise_df_502.csv")

summary_df, activities_df, profile_df = load_persona(
    summary_path=summary_path,
    activities_path=activities_path,
    enforce_schema=True,
    temporally_localize="today"
)

print(f"✓ Data loaded:")
print(f"  Summary: {summary_df.shape}")
print(f"  Activities: {activities_df.shape}")
print(f"  Profile: {profile_df.shape}")

# Show sample data
display(summary_df.head(2))

In [ ]:
# Cell 5: Create Agent
exemplar_paths = glob.glob(os.path.join("few_shots", "*.ipynb"))
print(f"Found {len(exemplar_paths)} exemplar notebooks")

agent = get_react_agent(
    summary_df=summary_df,
    activities_df=activities_df,
    profile_df=profile_df,
    example_files=exemplar_paths,
    tavily_api_key=TAVILY_API_KEY,
    use_mock_search=use_mock_search
)
print("✓ Agent created successfully!")

In [ ]:
# Cell 6: Ask a Question
question = "How can I feel more awake and energetic during the day?"
print(f"Question: {question}")
print("-" * 50)

full_question = QUESTION_PREFIX + question
final_answer, final_state = ot.run(
    agent(inputs=full_question, return_final_state=True)
)

print("\n💡 Answer:")
print("=" * 50)
print(final_answer)